# 特征降维:特征选择
- 学习目标
    - 了解降维的定义
    - 知道通过低方差过滤实现降维过程
    - 知道相关系数实现降维的过程
    - 知道主成分分析法实现过程
    

> 降维是指在某些限定条件下，降低随机变量(特征)个数，得到一组“不相关”主变量的过程,降低随机变量的个数

- 降维的两种方式
    - 特征选择
    - 主成分分析（可以理解一种特征提取的方式）


## 1.特征选择

> 1.1 定义:数据中包含冗余或无关变量（或称特征、属性、指标等），旨在从原有特征中找出主要特征。


1.2 方法
- Filter(过滤式)：主要探究特征本身特点、特征与特征和目标值之间关联
    - 方差选择法：低方差特征过滤
    - 相关系数
- Embedded (嵌入式)：算法自动选择特征（特征与目标值之间的关联）
    - 决策树:信息熵、信息增益
    - 正则化：L1、L2
    - 深度学习：卷积等
    
    
- 1.3 低方差特征过滤
    - 特征方差小：某个特征大多样本的值比较相近
    - 特征方差大：某个特征很多样本的值都有差别
    
    
- 1.3.1 API
    - sklearn.feature_selection.VarianceThreshold(threshold = 0.0)
        - 删除所有低方差特征
        - Variance.fit_transform(X)
        - X:numpy array格式的数据[n_samples,n_features]
        - 返回值：训练集差异低于threshold的特征将被删除。默认值是保留所有非零方差特征，即删除所有样本中具有相同值的特征。
        


In [9]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
data = pd.read_csv('../data/factor_returns.csv')
print(data.shape)
data.head()

(2318, 12)


,index,pe_ratio,pb_ratio,market_cap,return_on_asset_net_profit,du_return_on_equity,ev,earnings_per_share,revenue,total_expense,date,return
0,000001.XSHE,5.9572,1.1818,8.525255e+10,0.8008,14.9403,1.211445e+12,2.010,2.070140e+10,1.088254e+10,2012-01-31,0.027657
1,000002.XSHE,7.0289,1.5880,8.411336e+10,1.6463,7.8656,3.002521e+11,0.326,2.930837e+10,2.378348e+10,2012-01-31,0.082352
2,000008.XSHE,-262.7461,7.0003,5.170455e+08,-0.5678,-0.5943,7.705178e+08,-0.006,1.167983e+07,1.203008e+07,2012-01-31,0.099789
3,000060.XSHE,16.4760,3.7146,1.968046e+10,5.6036,14.6170,2.800916e+10,0.350,9.189387e+09,7.935543e+09,2012-01-31,0.121595
4,000069.XSHE,12.5878,2.5616,4.172721e+10,2.8729,10.9097,8.124738e+10,0.271,8.951453e+09,7.091398e+09,2012-01-31,-0.002681


In [14]:
def variance_demo(data, var_thread):
    """
    删除低方差特征——特征选择
    :return: None
    """
    # 1、实例化一个转换器类,删除方差小于threshold的属性
    transfer = VarianceThreshold(threshold=var_thread)
    # 2、调用fit_transform
    transfer_data = transfer.fit_transform(data.iloc[:, 1:10])
    print('降维前 shape:', data.iloc[:, 1:10].shape)
    print("删除低方差特征的结果：\n", transfer_data)
    print("降维后形状：\n", transfer_data.shape)

    return None

In [15]:

variance_demo(data,10)

降维前 shape: (2318, 9)
删除低方差特征的结果：
 [[ 5.95720000e+00  8.52525509e+10  8.00800000e-01 ...  1.21144486e+12
   2.07014010e+10  1.08825400e+10]
 [ 7.02890000e+00  8.41133582e+10  1.64630000e+00 ...  3.00252062e+11
   2.93083692e+10  2.37834769e+10]
 [-2.62746100e+02  5.17045520e+08 -5.67800000e-01 ...  7.70517753e+08
   1.16798290e+07  1.20300800e+07]
 ...
 [ 3.95523000e+01  1.70243430e+10  3.34400000e+00 ...  2.42081699e+10
   1.78908166e+10  1.74929478e+10]
 [ 5.25408000e+01  3.28790988e+10  2.74440000e+00 ...  3.88380258e+10
   6.46539204e+09  6.00900728e+09]
 [ 1.42203000e+01  5.91108572e+10  2.03830000e+00 ...  2.02066110e+11
   4.50987171e+10  4.13284212e+10]]
降维后形状：
 (2318, 7)


## 1.4 相关系数
- 皮尔逊相关系数
- 斯皮尔曼相关系数

- 1.4.1 皮尔逊相关系数(Pearson Correlation Coefficient)
    - 反映变量之间相关关系密切程度的统计指标
![avatar](../source/135.jpg)
- 当r>0时，表示两变量正相关，r<0时，两变量为负相关
- 当|r|=1时，表示两变量为完全相关，当r=0时，表示两变量间无相关关系
- |r|<0.4为低度相关；0.4≤|r|<0.7为显著性相关；0.7≤|r|<1为高度线性相关


- from scipy.stats import pearsonr
    - x : (N,) array_like
    - y : (N,) array_like Returns: (Pearson’s correlation coefficient, p-value)


In [17]:
from scipy.stats import pearsonr

x1 = [12.5, 15.3, 23.2, 26.4, 33.5, 34.4, 39.4, 45.2, 55.4, 60.9]
x2 = [21.2, 23.9, 32.9, 34.1, 42.5, 43.2, 49.0, 52.8, 59.4, 63.5]
#返回两个值,第一个是pearsonr 系数,约接近 1 越好,第二个是另外一个参数,当数组长度多余 500 的时候有参考意义,它约接近 0 约好
pearsonr(x1, x2)

(0.9941983762371883, 4.9220899554573455e-09)

## 1.4.2 斯皮尔曼相关系数(Rank IC)

- 反映变量之间相关关系密切程度的统计指标
![avatar](../source/136.jpg)

- 斯皮尔曼相关系数表明 X (自变量) 和 Y (因变量)的相关方向。 如果当X增加时， Y 趋向于增加, 斯皮尔曼相关系数则为正
- 与之前的皮尔逊相关系数大小性质一样，取值 [-1, 1]之间
- 斯皮尔曼相关系数比皮尔逊相关系数应用更加广泛

In [18]:
from scipy.stats import spearmanr

x1 = [12.5, 15.3, 23.2, 26.4, 33.5, 34.4, 39.4, 45.2, 55.4, 60.9]
x2 = [21.2, 23.9, 32.9, 34.1, 42.5, 43.2, 49.0, 52.8, 59.4, 63.5]

spearmanr(x1, x2)

SpearmanrResult(correlation=0.9999999999999999, pvalue=6.646897422032013e-64)

# PCA
- 定义：高维数据转化为低维数据的过程，在此过程中可能会舍弃原有数据、创造新的变量
- 作用：是数据维数压缩，尽可能降低原数据的维数（复杂度），损失少量信息。
- 应用：回归分析或者聚类分析当中





- sklearn.decomposition.PCA(n_components=None)
    - n_components:
        - 小数：表示保留百分之多少的信息
        - 整数：减少到多少特征
    - PCA.fit_transform(X) X:numpy array格式的数据[n_samples,n_features]
    - 返回值：转换后指定维度的array


In [19]:
from sklearn.decomposition import PCA
data = [[2,8,4,5],
[6,3,0,8],
[5,4,9,1]]


In [20]:
# 1、实例化PCA, 小数——保留多少信息
transfer = PCA(n_components=0.9)
# 2、调用fit_transform
data1 = transfer.fit_transform(data)

print("保留90%的信息，降维结果为：\n", data1)

保留90%的信息，降维结果为：
 [[ 1.28620952e-15  3.82970843e+00]
 [ 5.74456265e+00 -1.91485422e+00]
 [-5.74456265e+00 -1.91485422e+00]]


In [22]:
# 1、实例化PCA, 整数——指定降维到的维数
transfer2 = PCA(n_components=3)
# 2、调用fit_transform
data2 = transfer2.fit_transform(data)
data2

array([[ 1.28620952e-15,  3.82970843e+00,  5.26052119e-16],
       [ 5.74456265e+00, -1.91485422e+00,  5.26052119e-16],
       [-5.74456265e+00, -1.91485422e+00,  5.26052119e-16]])

- 降维的定义【了解】
    - 就是改变特征值，选择哪列保留，哪列删除
    - 目标是得到一组”不相关“的主变量
- 降维的两种方式【了解】
    - 特征选择
    - 主成分分析（可以理解一种特征提取的方式）
- 特征选择【知道】
    - 定义：提出数据中的冗余变量
        - 方法：
            - Filter(过滤式)：主要探究特征本身特点、特征与特征和目标值之间关联
                - 方差选择法：低方差特征过滤
                - 相关系数
            - Embedded (嵌入式)：算法自动选择特征（特征与目标值之间的关联）
                - 决策树:信息熵、信息增益
                - 正则化：L1、L2
- 低方差特征过滤【知道】
    - 把方差比较小的某一列进行剔除
    - api:sklearn.feature_selection.VarianceThreshold(threshold = 0.0)
        - 删除所有低方差特征
        - 注意，参数threshold一定要进行值的指定
- 相关系数【掌握】
    - 主要实现方式：
        - 皮尔逊相关系数
        - 斯皮尔曼相关系数
- 皮尔逊相关系数
    - 通过具体值的大小进行计算
    - 相对复杂
    - api:from scipy.stats import pearsonr
    - 返回值，越接近|1|，相关性越强；越接近0，相关性越弱
- 斯皮尔曼相关系数
    - 通过等级差进行计算
    - 比上一个简单
    - api:from scipy.stats import spearmanr
    - 返回值，越接近|1|，相关性越强；越接近0，相关性越弱
- pca【知道】
    - 定义：高维数据转换为低维数据，然后产生了新的变量
    - api:sklearn.decomposition.PCA(n_components=None)
        - n_components
        - 整数 -- 表示降低到几维
        - 小数 -- 保留百分之多少的信息